# Fine-tune ruBERT-tiny2

Загружаем библиотеки

In [1]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
import nltk
from transformers import AutoModel, AutoTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric, Dataset
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from io import StringIO

Загружаем отобранные статьи. Статьи из https://ria.ru/export/rss2/archive/index.xml, которые в течение трех суток были изменены

In [2]:
with open('datanews.json', encoding="utf-8") as f:
    read_data = f.read()
read_data = read_data.replace('\n][\n', ',\n')
articles = pd.read_json(StringIO(read_data), orient='records')

Формируем предложения из проверенных дважы статей. Второе предложение типа "МОСКВА, 12 янв – РИА Новости" убираем.
В новый DataFrame записываем предложение и признак, что предложения корректые (правильные)

In [3]:
pst = nltk.PunktSentenceTokenizer()
sentences = pd.DataFrame({"sentence": [], "label": []})
for ind in articles.index:
    sentArticle = pst.tokenize(articles['Article'][ind])
    m = 0
    for s in sentArticle:
        if m != 1:
            sentences.loc[len(sentences.index)] = [s, 1]
print("Number of correct sentences: ", len(sentences.index))
sentences.head(3)

Number of correct sentences:  927


,sentence,label
0,В Госдуме предложили сделать старый Новый год ...,1
1,Председатель союза дачников Подмосковья и депу...,1
2,"""После длинных новогодних праздников людям тяж...",1


Добавляем предложения с ошибками. При этом устанавливаем признак, что предложения неправильные

In [4]:
error_list = []
with open('errorsents.txt', encoding="utf-8") as fe:
    error_list = fe.readlines()
for s in error_list:
    sentences.loc[len(sentences.index)] = [s, 0]
print("Number of incorrect sentences: ", len(error_list))

Number of incorrect sentences:  5


Формируем набор для обучения и тестовый набор

In [5]:
train, test = train_test_split(sentences, test_size=0.2, random_state=42)
train_text = train['sentence'].astype('str')
test_text = test['sentence'].astype('str')
train_labels = train['label']
test_labels = test['label']


Задание всех seed

In [6]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(42)

In [7]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

In [15]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.values.tolist(),
    padding = True,
    truncation = True,
    add_special_tokens = True,
    return_attention_mask = True,
    return_tensors = 'pt'
)
tokens_test = tokenizer.batch_encode_plus(
    test_text.values.tolist(),
    padding = True,
    truncation = True,
    add_special_tokens = True,
    return_attention_mask = True,
    return_tensors = 'pt'
)

In [9]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
    def __len__(self):
        return len(self.labels)
    
train_dataset = Data(tokens_train, train_labels)
test_dataset = Data(tokens_test, test_labels)

https://www.javatpoint.com/accuracy_score-in-sklearn

In [16]:
def compute_accuracy(Y_true, Y_pred):  
    correctly_predicted = 0  
    # iterating over every label and checking it with the true sample  
    for true_label, predicted in zip(Y_true, Y_pred):  
        if true_label == predicted:  
            correctly_predicted += 1  
    # computing the accuracy score  
    accuracy_score = correctly_predicted / len(Y_true)  
    return accuracy_score  

# Training the model using the Support Vector Classification class of sklearn  
svc = SVC()  
svc.fit(tokens_train['input_ids'], train_labels)  
  
# Computing the accuracy score of the model  
Y_pred = svc.predict(tokens_train['input_ids'])  
score = compute_accuracy(test_labels, Y_pred)  
print(score)  

0.9893048128342246


Функция для расчета метрики. Используется метрику F1, так как классы не сбалансированы:

In [17]:
from sklearn.metrics import f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds)
    return {'F1': f1}

Ниже указаны все параметры, которые будут использоваться для обучения:

In [18]:
training_args = TrainingArguments(
    output_dir = './results', #Выходной каталог
    num_train_epochs = 3, #Кол-во эпох для обучения
    per_device_train_batch_size = 8, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 8, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = './logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=21)

Передача в trainer предообученную модель, tokenizer, данные для обучения, данные для валидации и способ расчета метрики:

In [13]:
trainer = Trainer(model=model,
                  tokenizer = tokenizer,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = train_dataset,
                  compute_metrics = compute_metrics)

Запуск обучения модели:

In [19]:
trainer.train()

AttributeError: 'AcceleratorState' object has no attribute 'distributed_type'

Сохранение обученной модели:

model_path = "fine-tune-rubert-tiny2"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Написание функции для получения предикта:

In [ ]:
def get_prediction():
    test_pred = trainer.predict(test_dataset)
    labels = np.argmax(test_pred.predictions, axis = -1)
    return labels
pred = get_prediction()

Вывод всей необходимой информации для оценки качества модели:

In [ ]:
print(classification_report(test_labels, pred))
print(f1_score(test_labels, pred))